In [1]:
# setup
import os
import pandas as pd
from datetime import datetime
import numpy as np
import utils

users = pd.read_csv(
        'https://raw.githubusercontent.com/treehouse-projects/python-introducing-pandas/master/data/users.csv' , index_col=0)
transactions = pd.read_csv('https://raw.githubusercontent.com/treehouse-projects/python-introducing-pandas/master/data/transactions.csv', index_col = 0)
requests = pd.read_csv('https://raw.githubusercontent.com/treehouse-projects/python-introducing-pandas/master/data/requests.csv',
                      index_col = 0)

succesful_requests = requests.merge(
    transactions,
    left_on = ['from_user', 'to_user', 'amount'],
    right_on = ['receiver', 'sender', 'amount']
)

succesful_requests['request_date'] = pd.to_datetime(succesful_requests['request_date'])
succesful_requests['sent_date'] = pd.to_datetime(succesful_requests['sent_date'])
succesful_requests['timedelta'] = succesful_requests.sent_date - succesful_requests.request_date

# Sense check
succesful_requests.sort_values(by = 'timedelta', ascending = False).head()


,from_user,to_user,amount,request_date,sender,receiver,sent_date,timedelta
0,chad.chen,paula7980,78.61,2018-02-12,paula7980,chad.chen,2018-07-15,153 days
33,sthompson,andrade,14.07,2018-05-09,andrade,sthompson,2018-09-21,135 days
4,lacey8987,mcguire,54.09,2018-03-13,mcguire,lacey8987,2018-06-28,107 days
53,marcus.berry,melissa.mendoza,71.48,2018-05-31,melissa.mendoza,marcus.berry,2018-09-06,98 days
39,bishop,massey2102,18.27,2018-05-16,massey2102,bishop,2018-08-15,91 days


## Further Exploration

Because you can request for money multiple times there can be duplicates. This is because if money is requested 3 times and sent once the sent will match to all 3 requests.

We will now explore more advanced techniques for how to deal with this

In [2]:
# Create a boolean Series of records that are duplicated, note: keep=False marks all that are duplicates
dupes = requests[requests.duplicated(('from_user', 'to_user', 'amount'),keep = False)]
# Order by requester and date of requests
dupes.sort_values(['from_user', 'request_date']).head()

,from_user,to_user,amount,request_date
58,austin486,shelly,11.24,2018-05-29
59,austin486,shelly,11.24,2018-05-29
8,cjimenez,sarah.evans,48.14,2018-03-21
26,cjimenez,sarah.evans,48.14,2018-04-27
218,clark8139,moore,14.54,2018-08-31


To see the actual values of all the transactions we can drop duplicates from our DataFrame

In [3]:
# Sort our transactions chronologically
succesful_requests.sort_values('request_date', inplace = True)

# We can then drop dupes keeping only the latest transaction
succesful_requests.drop_duplicates(('from_user', 'to_user', 'amount'), keep = 'last', inplace = True)
"Wow, we have had £{:,.2f} transactions!".format(
    succesful_requests.amount.sum(),
    len(succesful_requests))

'Wow, we have had £9,316.12 transactions!'

## Locating Records not Found in Another Dataset

Now that we have seen how succesful the requests have been, it would be nice to see which users haven't made a request to another user.

Basically we want to see who is in the users DataFrame and not the requests DataFrame

In [4]:
# Create a boolean array where we can check if the label (username) is in the from_user Series
made_request_index = users.index.isin(requests.from_user)
# This will give a list of all user who have made a request
users[made_request_index].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
aaron,Aaron,Davis,aaron6348@gmail.com,True,2018-08-31,6,18.14
acook,Anthony,Cook,cook@gmail.com,True,2018-05-12,2,55.45
adam.saunders,Adam,Saunders,adam@gmail.com,False,2018-05-29,3,72.12
adrian,Adrian,Fang,adrian.fang@teamtreehouse.com,True,2018-04-28,3,30.01
adrian.blair,Adrian,Blair,adrian9335@gmail.com,True,2018-06-16,7,25.85


Because we have a boolean index we can use the `~` bitwise operator to get what we're looking for, users who have no made a request

In [5]:
users[~made_request_index].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
alan9443,Alan,Pope,pope@hotmail.com,True,2018-04-17,0,56.09
alvarado,Denise,Alvarado,alvarado@hotmail.com,True,2018-09-07,6,26.72
amiller,Anne,Miller,miller@hotmail.com,False,2018-06-02,5,86.28
andersen,Mark,Andersen,mark.andersen@yahoo.com,True,2018-08-21,3,75.69
andrade,Melissa,Andrade,mandrade@yahoo.com,True,2018-01-06,3,83.22


## Locating Missing Data

As we was first looking at the data we checked the count for each column, from this we can see if there's missing values

In [6]:
users.count()

first_name        475
last_name         430
email             475
email_verified    475
signup_date       475
referral_count    475
balance           475
dtype: int64

`Series.isna` is a method that that allows you to check for missing values. It returns a boolean array which can be used as an index

In [7]:
users[users.last_name.isna()].head()

,first_name,last_name,email,email_verified,signup_date,referral_count,balance
anthony3761,Anthony,NaN,anthony9581@gmail.com,True,2018-08-16,6,59.81
bradley,Bradley,NaN,bradley3941@henderson.com,True,2018-06-24,4,22.24
bradley7808,Bradley,NaN,bradley8794@yahoo.com,True,2018-06-15,1,76.49
brenda,Brenda,NaN,brenda@hotmail.com,True,2018-07-28,7,6.18
brooke2027,Brooke,NaN,brooke6938@gmail.com,False,2018-05-23,0,7.22


### Filling Missing Data

You can also set all unknown values to a specific value using the `fillna` method. By default it will return a new DataFrame but we can use `inplace` to put it in the same DataFrame

In [8]:
# Fill the missing values
users_with_unknown = users.fillna('Unknown')

# Make sure we've got them all
users_with_unknown[users_with_unknown.last_name.isna()]

,first_name,last_name,email,email_verified,signup_date,referral_count,balance


### Dropping Rows With Missing Data

Another option is to just drop data with missing values. That is the `DataFrame.dropna` method. Use `inplace` to update the existing `DataFrame`

In [9]:
users_with_last_names = users.dropna()
# compare sizes
(len(users), len(users_with_last_names))

(475, 430)

## Manipulating Text

Oftentimes, there will be something a bit off with the string data in your dataset. You may want to replace some characters, change the case or strip the whitespace. 

We can do this by using a loop but a faster and easier way is to use __vectorisation__

In [10]:
pd.options.display.max_rows = 10
def make_chaos(df, sample_size, columns, fn):
    # Keep chaos the same randomly
    some = df.sample(sample_size, random_state=sample_size)
    for col in columns:
        some[col] = some[col].apply(fn)
    # Update the original DataFrame
    df.update(some)
make_chaos(transactions, 42, ['sender'], lambda val: '$' + val)
make_chaos(transactions, 88, ['receiver'], lambda val: val.upper())

### Replacing Text

When CashBox first started, usernames with `$` were allowed. As time progressed, they changed their mind. They made a mass update to the system. However, someone in the customer service team reported that they can still see some usernames in the transactions table starting with `$`. 

To get these we can use the `Series.str.startswith()` method. This returns a `boolean` series which can be used as an index.

In [11]:
transactions[transactions.sender.str.startswith('$')]

,sender,receiver,amount,sent_date
59,$porter,gail7896,75.16,2018-05-14
70,$emily.lewis,kevin,5.49,2018-05-21
158,$robinson,rodriguez,8.91,2018-06-25
168,$nancy,margaret265,84.15,2018-06-26
198,$acook,adam.saunders,9.31,2018-07-04
...,...,...,...,...
877,$april9082,jacob.davis,50.37,2018-09-21
889,$victor,anthony1788,39.06,2018-09-21
900,$andersen,corey.ingram,4.81,2018-09-22
927,$janet.williams,bsmith,50.15,2018-09-23


We can now replace all the `$`'s with an empty string, essentially removing all the `$`'s. to do this we can use the `Series.str.replace()` method. 

In [12]:
transactions.sender = transactions.sender.str.replace('$','')
# make sure we've got them all
len(transactions[transactions.sender.str.startswith('$')])

0

### Changing Case

When you want to select or merge by specific values the case of the strings matters. For CashBox all the username's should be in lowercase but we have noticed they are not. 

We can get a handle on them by using `Series.str.isupper()` to get a handle on those cases which contain upper case characters. This will return a `boolean` series which can be used as an index. 

In [13]:
transactions[transactions.receiver.str.isupper()]

,sender,receiver,amount,sent_date
2,rose.eaton,EMILY.LEWIS,62.67,2018-02-15
5,francis.hernandez,LMOORE,91.46,2018-03-14
14,palmer,CHAD.CHEN,36.27,2018-04-07
28,elang,DONNA1922,26.07,2018-04-23
34,payne,GRIFFIN4992,85.21,2018-04-26
...,...,...,...,...
963,stanley7729,JOSEPH.LOPEZ,50.84,2018-09-25
977,martha6969,PATRICIA,87.33,2018-09-25
987,alvarado,PAMELA,48.74,2018-09-25
990,robert,HEATHER.WADE,86.44,2018-09-25


From this we can use the `Series.str.lower()` vectorised method to replace the values that contain upper case

In [14]:
transactions.loc[transactions.receiver.str.isupper(), 'receiver'] = transactions.receiver.str.lower()
len(transactions[transactions.receiver.str.isupper()])

0

## Grouping

A common need is bound to arise when you need to look at an aggregate of a view of DataFrame by a certain value.

CashBox has asked that we produce a list of the top 10 users who have been on the receiving side of the transactions the most. They would like to see the users first name, last name, email and the total number of transactions where the user was the receiver. 

We can achieve this by grouping our data and we can do that using `GroupBy`. Grouping by a specific column is fairly simple and is done by using the `DataFrame.groupby` method

In [15]:
group_by_receiver = transactions.groupby('receiver')
# look at what object it returns
type(group_by_receiver)

pandas.core.groupby.groupby.DataFrameGroupBy

We receive a DataFrameGroupBy object which has quite a few methods. First we will look at the size of it

In [16]:
group_by_receiver.size()

receiver
aaron            6
acook            1
adam.saunders    2
adrian           3
adrian.blair     7
                ..
wilson           2
wking            2
wright3590       4
young            2
zachary.neal     4
Length: 410, dtype: int64

We can also look at the `.count()` method to see the counts of the non missing data points in the DataFrame

In [17]:
group_by_receiver.count()

,sender,amount,sent_date
receiver,,,
aaron,6,6,6
acook,1,1,1
adam.saunders,2,2,2
adrian,3,3,3
adrian.blair,7,7,7
...,...,...,...
wilson,2,2,2
wking,2,2,2
wright3590,4,4,4


The `GroupBy` object provides aggregate functions that makes getting caluclations quick and seamless. For instance we can use `GroupBy.sum` we can see each numeric column being summed up. In this case the only numeric column is _`amount`_

In [18]:
group_by_receiver.sum()

,amount
receiver,
aaron,366.15
acook,94.65
adam.saunders,101.15
adrian,124.36
adrian.blair,462.88
...,...
wilson,44.39
wking,74.07
wright3590,195.45


We can use grouping to create a new column on our `users` DataFrame

In [19]:
# creating the new column called transaction count and setting the values to the size of the matching group
users['transactions_count'] = group_by_receiver.size()

# not every user has made a transactions so lets see the missing data we are dealing with
len(users[users.transactions_count.isna()])

65

It looks like we have made some `np.nan`'s due to not everyone making a transaction. We can set these to equal 0

In [20]:
users.transactions_count.fillna(0, inplace = True)
users

,first_name,last_name,email,email_verified,signup_date,referral_count,balance,transactions_count
aaron,Aaron,Davis,aaron6348@gmail.com,True,2018-08-31,6,18.14,6.0
acook,Anthony,Cook,cook@gmail.com,True,2018-05-12,2,55.45,1.0
adam.saunders,Adam,Saunders,adam@gmail.com,False,2018-05-29,3,72.12,2.0
adrian,Adrian,Fang,adrian.fang@teamtreehouse.com,True,2018-04-28,3,30.01,3.0
adrian.blair,Adrian,Blair,adrian9335@gmail.com,True,2018-06-16,7,25.85,7.0
...,...,...,...,...,...,...,...,...
wilson,Robert,Wilson,robert@yahoo.com,False,2018-05-16,5,59.75,2.0
wking,Wanda,King,wanda.king@holt.com,True,2018-06-01,2,67.08,2.0
wright3590,Jacqueline,Wright,jacqueline.wright@gonzalez.com,True,2018-02-08,6,18.48,4.0
young,Jessica,Young,jessica4028@yahoo.com,True,2018-07-17,4,75.39,2.0


Finally we can sort to get the users with the top transaction counts

In [23]:
# Lets change the type from float to int to make things quicker
users.transactions_count = users.transactions_count.astype('int64')

# Sort descending on transactions_count and ascending on first_name
users.sort_values(
['transactions_count', 'first_name'],
    ascending = [False, True],
    inplace = True
)

# look at the top 10
users.loc[:, ['first_name', 'last_name', 'email', 'transactions_count']].head(10)

,first_name,last_name,email,transactions_count
scott3928,Scott,NaN,scott@yahoo.com,9
sfinley,Samuel,Finley,samuel@gmail.com,8
adrian.blair,Adrian,Blair,adrian9335@gmail.com,7
hdeleon,Hannah,Deleon,hannah@yahoo.com,7
miranda6426,Miranda,Rogers,miranda.rogers@gmail.com,7
aaron,Aaron,Davis,aaron6348@gmail.com,6
corey,Corey,Fuller,fuller8100@yahoo.com,6
heather,Heather,Ray,hray@yahoo.com,6
jennifer.hebert,Jennifer,Hebert,jennifer.hebert@yahoo.com,6
edwards,Michael,Edwards,edwards5456@gmail.com,6
